<a href="https://colab.research.google.com/github/karchx/ml_101/blob/main/nlp_polizas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 1s (4,339 kB/s)
Selecting previously unselected package popp

In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [3]:
import re
import PyPDF2

In [4]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text.replace("\n", " ").replace(' ', '')

In [6]:
text = extract_text_from_pdf("./395328-1-11-2.pdf")
print(text)

PólizaNo.INFA-1038SumaAsegurada:Desde:Hasta:ProintelCorredorDeSeguros,SociedadAnónima31/08/202431/08/202512:00A12:00HRSPolizaContratadaen:QuetzalesRamode:MultiriesgoFacultativoPrimaGtos.EmisiónGtos.Fracc.IVAOtrosGastosDto.1422TotalQQQQQQQ132,000,000.00Q.158,400.007,920.000.000.003,160.0019,958.40189,438.40SegurosG&T,S.A.DenominadaenadelantelaCompañia,condomicilioenlaciudaddeGuatemala,RepúblicadeGuatemala,conbaseydeacuerdoconlasdeclaracionesformuladasenlasolicitudrespectiva.ASEGURAA:CAMPUSTECNOLÓGICO,SOCIEDADANONIMADenominadoenadelanteelAsegurado,condomicilioen:Via41-00-Zona4-Guatemala-Guatemala-GuatemalaContraperdidaodañosalosbienesointeresesasegurados,causadosporlosriesgosindicadoscomocubiertosenlasCondicionesParticularesy/oEspecialesdelaPóliza.LaCompañíaconvieneconelAsegurado,sujetoalasCondicionesGenerales,Particularesy/oEspecialesdeestapólizayalasqueposteriormente,conaceptacióndelAseguradoopordisposiciónlegal,seagreguenmedianteelEndosooAnexo,queencasodesiniestroencualquierépocaduran

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModelForQuestionAnswering

tokenizer_ner = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model_ner = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp_ner = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

model_qa_name = "deepset/tinyroberta-squad2"
tokenizer_qa = AutoTokenizer.from_pretrained(model_qa_name)
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_qa_name)
nlp_qa = pipeline('question-answering', model=model_qa, tokenizer=tokenizer_qa)

def ask_question(question, context):
    QA_input = {
        'question': question,
        'context': context
    }
    result = nlp_qa(QA_input)
    return result['answer']

ner_results = nlp_ner(text)


nombre_cliente = ask_question('What is the client name?', text)
marca_vehiculo = ask_question('What is the car brand?', text)
chasis_vehiculo = ask_question('What is the chassis number?', text)
tipo_vehiculo = ask_question('What is the type of vehicle?', text)

print(f"Nombre del Cliente: {nombre_cliente}")
print(f"Marca del Vehículo: {marca_vehiculo}")
print(f"Chasis del Vehículo: {chasis_vehiculo}")
print(f"Tipo de Vehículo: {tipo_vehiculo}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/326M [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Nombre del Cliente: EXPLICAENESTA
Marca del Vehículo: EXPLICAENESTA
Chasis del Vehículo: Q.100,000.00
Tipo de Vehículo: RORISMO
